### Exercise 4.2: Sentiment Analysis

#### DSC 550

Taniya Adhikari 1/10/2021

In [553]:
import pandas as pd
import numpy as np
import re
from re import sub
import multiprocessing
from unidecode import unidecode

from gensim.models.phrases import Phrases, Phraser
from gensim.models import Word2Vec
from gensim.test.utils import get_tmpfile
from gensim.models import KeyedVectors


from time import time 
from collections import defaultdict

import logging  # Setting up the loggings to monitor gensim
logging.basicConfig(format="%(levelname)s - %(asctime)s: %(message)s", datefmt= '%H:%M:%S', level=logging.INFO)
import textblob

nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize
from textblob import TextBlob
from sklearn.cluster import KMeans

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\taniy\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


1. Load the data file DailyComments.csv from the Week 4 Data Files into a data frame.

In [554]:
df = pd.read_csv("DailyComments.csv")
df.count()

Day of Week    7
comments       7
dtype: int64

In [555]:
df.head(7)

,Day of Week,comments
0,Monday,"Hello, how are you?"
1,Tuesday,Today is a good day!
2,Wednesday,It's my birthday so it's a really special day!
3,Thursday,Today is neither a good day or a bad day!
4,Friday,I'm having a bad day.
5,Saturday,There' s nothing special happening today.
6,Sunday,Today is a SUPER good day!


Text Pre-processing before modeling for sentiment analysis

In [556]:
def pre_processing(df):
    # converting all text to lowercase
    df["comments"] = df["comments"].str.lower()
    
    # removing punctuation using string.punctuations and join()
    df["comments"] = df["comments"].apply(lambda x: "".join([i for i in x if i not in string.punctuation]))
    
    # remove stop words
    stop = stopwords.words('english')

    df['stopwords'] = df["comments"].apply(lambda x: len([i for i in x.split() if i in stop]))
    df["comments"]= df["comments"].apply(lambda x: " ".join(i for i in x.split() if i not in stop))
    df['stopwords'] = df['comments'].apply(lambda x: len([i for i in x.split() if i in stop]))
    
    # stemming
    porter = PorterStemmer()
    df["comments"] = df["comments"].apply(lambda x: " ".join([porter.stem(word) for word in x.split()]))
    return df

df2 = pre_processing(df)
df2

,Day of Week,comments,stopwords
0,Monday,hello,0
1,Tuesday,today good day,0
2,Wednesday,birthday realli special day,0
3,Thursday,today neither good day bad day,0
4,Friday,im bad day,0
5,Saturday,noth special happen today,0
6,Sunday,today super good day,0


In [557]:
# converting string into list of words for each comment
def text_to_list(text):
    pattern = r'[^A-Za-z ]'
    regex = re.compile(pattern)
    text = regex.sub('', text).split(' ')
    return text

In [558]:
clean_df = df2.copy()
clean_df.comments = clean_df.comments.apply(lambda x: text_to_list(x))
clean_df.head()

,Day of Week,comments,stopwords
0,Monday,[hello],0
1,Tuesday,"[today, good, day]",0
2,Wednesday,"[birthday, realli, special, day]",0
3,Thursday,"[today, neither, good, day, bad, day]",0
4,Friday,"[im, bad, day]",0


In [559]:
#extraction of phrases and bigram model
sent = [row for row in clean_df.comments]
phrases = Phrases(sent, min_count=1, progress_per=7)
bigram = Phraser(phrases)
sentences = bigram[sent]
sentences[6]

INFO - 14:04:02: collecting all words and their counts
INFO - 14:04:02: PROGRESS: at sentence #0, processed 0 words and 0 word types
INFO - 14:04:02: collected 28 word types from a corpus of 25 words (unigram + bigrams) and 7 sentences
INFO - 14:04:02: using 28 counts as vocab in Phrases<0 vocab, min_count=1, threshold=10.0, max_vocab_size=40000000>
INFO - 14:04:02: source_vocab length 28
INFO - 14:04:02: Phraser built with 0 phrasegrams


['today', 'super', 'good', 'day']

In [560]:
w2v_model = Word2Vec(min_count=1, size=2)

# building vocab
w2v_model.build_vocab(sentences, progress_per=7)

WARNING - 14:04:04: consider setting layer size to a multiple of 4 for greater performance
INFO - 14:04:04: collecting all words and their counts
INFO - 14:04:04: PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
INFO - 14:04:04: collected 13 word types from a corpus of 25 raw words and 7 sentences
INFO - 14:04:04: Loading a fresh vocabulary
INFO - 14:04:04: effective_min_count=1 retains 13 unique words (100% of original 13, drops 0)
INFO - 14:04:04: effective_min_count=1 leaves 25 word corpus (100% of original 25, drops 0)
INFO - 14:04:04: deleting the raw counts dictionary of 13 items
INFO - 14:04:04: sample=0.001 downsamples 13 most-common words
INFO - 14:04:04: downsampling leaves estimated 3 word corpus (12.1% of prior 25)
INFO - 14:04:04: estimated required memory for 13 words and 2 dimensions: 6708 bytes
INFO - 14:04:04: resetting layer weights


In [561]:
# training the model
w2v_model.train(sentences, total_examples=w2v_model.corpus_count, epochs=4, report_delay=1)
w2v_model.init_sims(replace=True)

INFO - 14:04:04: training model with 3 workers on 13 vocabulary and 2 features, using sg=0 hs=0 sample=0.001 negative=5 window=5
INFO - 14:04:04: worker thread finished; awaiting finish of 2 more threads
INFO - 14:04:04: worker thread finished; awaiting finish of 1 more threads
INFO - 14:04:04: worker thread finished; awaiting finish of 0 more threads
INFO - 14:04:04: EPOCH - 1 : training on 25 raw words (1 effective words) took 0.0s, 324 effective words/s
INFO - 14:04:04: worker thread finished; awaiting finish of 2 more threads
INFO - 14:04:04: worker thread finished; awaiting finish of 1 more threads
INFO - 14:04:04: worker thread finished; awaiting finish of 0 more threads
INFO - 14:04:04: EPOCH - 2 : training on 25 raw words (1 effective words) took 0.0s, 376 effective words/s
INFO - 14:04:04: worker thread finished; awaiting finish of 2 more threads
INFO - 14:04:04: worker thread finished; awaiting finish of 1 more threads
INFO - 14:04:04: worker thread finished; awaiting finish 

In [562]:
print(w2v_model)

Word2Vec(vocab=13, size=2, alpha=0.025)


In [563]:
w2v_model.save("word2vec.model")

INFO - 14:04:06: saving Word2Vec object under word2vec.model, separately None
INFO - 14:04:06: not storing attribute vectors_norm
INFO - 14:04:06: not storing attribute cum_table
INFO - 14:04:06: saved word2vec.model


In [564]:
word_vectors = Word2Vec.load("word2vec.model").wv

INFO - 14:04:07: loading Word2Vec object from word2vec.model
INFO - 14:04:07: loading wv recursively from word2vec.model.wv.* with mmap=None
INFO - 14:04:07: setting ignored attribute vectors_norm to None
INFO - 14:04:07: loading vocabulary recursively from word2vec.model.vocabulary.* with mmap=None
INFO - 14:04:07: loading trainables recursively from word2vec.model.trainables.* with mmap=None
INFO - 14:04:07: setting ignored attribute cum_table to None
INFO - 14:04:07: loaded word2vec.model


In [565]:
# k-means clustering to create two centroids which will later used as positive negative sentiments
model = KMeans(n_clusters=2, max_iter=100).fit(X=word_vectors.vectors.astype("double"))

In [566]:
model.cluster_centers_

array([[-0.78832501, -0.0736144 ],
       [ 0.61350253,  0.39413982]])

In [567]:
# check for similarity
word_vectors.similar_by_vector(model.cluster_centers_[0], restrict_vocab=None)

INFO - 14:04:12: precomputing L2-norms of word weight vectors


[('good', 0.9780603647232056),
 ('hello', 0.9035085439682007),
 ('super', 0.9016662836074829),
 ('realli', 0.8192667365074158),
 ('noth', 0.8170955181121826),
 ('birthday', 0.7565217614173889),
 ('day', 0.5820415019989014),
 ('special', 0.5758768320083618),
 ('neither', -0.3850495517253876),
 ('today', -0.45182546973228455)]

In [568]:
positive = model.cluster_centers_[0]
negative = model.cluster_centers_[1]

In [569]:
words = pd.DataFrame(word_vectors.vocab.keys())
words.columns = ['words']
words['vectors'] = words.words.apply(lambda x: word_vectors[f'{x}'])
words['cluster'] = words.vectors.apply(lambda x: model.predict([np.array(x)]))
words.cluster = words.cluster.apply(lambda x: x[0])

In [570]:
words['cluster_value'] = [1 if i==0 else -1 for i in words.cluster]
words['closeness_score'] = words.apply(lambda x: 1/(model.transform([x.vectors]).min()), axis=1)
words['sentiment_coeff'] = words.closeness_score * words.cluster_value

words

,words,vectors,cluster,cluster_value,closeness_score,sentiment_coeff
0,hello,"[-0.9394416, 0.34270886]",0,1,2.257841,2.257841
1,today,"[0.532813, -0.846233]",1,-1,0.804509,-0.804509
2,good,"[-0.95445466, -0.29835606]",0,1,3.578101,3.578101
3,day,"[-0.5039158, -0.86375284]",0,1,1.190808,1.190808
4,birthday,"[-0.6924411, -0.7214743]",0,1,1.526911,1.526911
5,realli,"[-0.86903167, 0.4947565]",0,1,1.741941,1.741941
6,special,"[-0.64939374, 0.76045233]",0,1,1.182650,1.182650
7,neither,"[0.29757416, 0.9546987]",1,-1,1.554105,-1.554105
8,bad,"[0.63992137, 0.7684404]",1,-1,2.665019,-2.665019
9,im,"[0.6566255, 0.7542168]",1,-1,2.757480,-2.757480


To do a simple sentiment analysis for small data set with few words, I decided to focus on each word's sentiment instead of sentiment_coefficient.

Following code, takes word list from each comment, looks up word in words dataframe and retreives cluster_value which is the sentiment of the word. It creates a dictionary with the counts of positive and negative words for each comments and then compares the two values. This returns the sentiment of each comment. 

In [571]:
def sentiment_analyzer(wordlist):
    cluster_count = {}
    for w in wordlist:
        x = cluster_freq(w, words)
        if x in cluster_count:
            cluster_count[x] = cluster_count[x] + 1
        else:
            cluster_count[x] = 1
                  
    if '-1' in cluster_count.keys() and '1' in cluster_count.keys():
        pos = cluster_count.get('1')
        neg = cluster_count.get('-1')
        if pos >= neg:
            sentiment = "Positive"
        else:
            sentiment = "Negative"

    elif '-1' not in cluster_count.keys() and '1' in cluster_count.keys():
        sentiment = "Positive"
    elif '1' not in cluster_count.keys() and '-1' in cluster_count.keys():
        sentiment = "Negative"
    else:
        None
    return sentiment

In [572]:
def cluster_freq(word, df):
    x = ''
    for i, r in df.iterrows():
        if r['words'] == word:
            x = str(r['cluster_value'])
        else:
            None
    return x
        

Following code iterate through each row of the df and turn comments into string and then list of words.
Further it's passsed to function sentiment_analyzer

In [573]:
cluster_count = {}
sentiment = []
for i, r in df2.iterrows():
    comment = r['comments']
    wordlist = comment.split()
    result = sentiment_analyzer(wordlist)
    sentiment.append(result)
df2['sentiment'] = sentiment    

df2

,Day of Week,comments,stopwords,sentiment
0,Monday,hello,0,Positive
1,Tuesday,today good day,0,Positive
2,Wednesday,birthday realli special day,0,Positive
3,Thursday,today neither good day bad day,0,Positive
4,Friday,im bad day,0,Negative
5,Saturday,noth special happen today,0,Positive
6,Sunday,today super good day,0,Positive


In [574]:
def sentiment_func(comments):
    pol = TextBlob(comments).sentiment.polarity
    if pol >= 0:
        x = "Positive"
    else:
        x = "Negative"
    return x

In [575]:
df2['textblob_sentiment']= df2['comments'].apply(sentiment_func)

In [576]:
df2

,Day of Week,comments,stopwords,sentiment,textblob_sentiment
0,Monday,hello,0,Positive,Positive
1,Tuesday,today good day,0,Positive,Positive
2,Wednesday,birthday realli special day,0,Positive,Positive
3,Thursday,today neither good day bad day,0,Positive,Positive
4,Friday,im bad day,0,Negative,Negative
5,Saturday,noth special happen today,0,Positive,Positive
6,Sunday,today super good day,0,Positive,Positive


The data didn't have sentiment associated with it, so I decided to take unsupervised approach for a small dataset. I used word2vec model, kmeans clustering algorithm and my own algorithm to do my own analysis. For Word2vec we used CBOW architecture code can be found here [myGitHub](https://github.com/adhtani/DSC550/blob/master/Exercise4_TAdhikari.ipynb). I also followed procedure from this tutorial [Unsupervised Sentiment Analysis](https://towardsdatascience.com/unsupervised-sentiment-analysis-a38bf1906483). I created bigrams of words detection and replaced with gensim’s Phrases module (see In [559]) and used that to create word2vec model, further I build a vocabulary and trained the model.

These are initial steps before kmeans clustering algorithm. I used sklearn’s K-means algorithm with 100 iterations of reassigning points to clusters and used 2 clusters. I checked with gensim’s most_similar method to see which word vectors are similar to first cluster. I created a df with words and their sentiment coefficient.

Furthermore, I created a simple algorithm that will count the positive and negative words in each comments. and assign sentiments based on number of positive and negative words see [571, 572, 573]).
I further checked the accuracy based on results from Textblob which is another python library that has a pre-trained dataset and uses Naive bayes classifier to predict sentiment.

Results were fairly accurate based on the assigned positive and negative values. However, simple counting good and bad words is not the best approach because sentiment analysis works by looking at the pattern within the data. This will not work when the dataset is big with complex texts such as tweets. In my extra exercise file I tried this method with 20 stocktwits data, and I got inaccurate results. So for the bigger dataset **Tfidf weighting** for sentiment prediction will work better with complex data.